In [770]:
import pandas as pd
import numpy as np
import math

df = pd.read_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\BGB-16673-102_V1.0.xlsx', sheet_name='CheckSteps')
df

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,NaN
1,DYN_SV_SET_SVDELYN_VISIBLE,2,NaN,Done,$5,...,NaN,NaN,NaN,NaN,NaN
2,DYN_SV_SET_SVDELYN_VISIBLE,3,IsEqualTo,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
3,DYN_SV_SET_SVDRES_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,NaN
4,DYN_SV_SET_SVDRES_VISIBLE,2,NaN,Done,$10,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13614,Chk_SCR_DS_IC_ICOBM_MHTERM8,8,IsNotEmpty,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
13615,Chk_SCR_DS_IC_ICOBM_MHTERM8,9,And,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
13616,Chk_SCR_DS_IC_ICOBM_MHTERM8,10,And,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
13617,Chk_CM_POST_CMPDDAT4_02_b_CF,1,NaN,NaN,DataPoint,...,NaN,NaN,NaN,NaN,NaN


In [771]:
pd.set_option('display.max_rows', 3)
pd.set_option('display.max_columns', 10)

In [772]:
# data clean
df = df[~df['CheckName'].str.contains('RSG')]
df

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13618,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [773]:
def isFormRpt(x):
    if x == 'CM_PST':
        result = 1
    elif x == 'AE_MIX':
        result = 1
    elif x == 'CM_POST':
        result = 1
    else:
        result = 0
    return result

df['FormRpt'] = df['FormOID'].apply(isFormRpt)
df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
13618,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,NaN,0


In [774]:
df_sum = df.groupby('CheckName').sum()
df_sum = pd.DataFrame(df_sum,columns=['FormRpt'])
df_sum

,FormRpt
CheckName,
CF_DER_LABAGE_A,0
...,...
Update_AllInstanceNames_Empty_CF,0


In [775]:
df_sum.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\df_sum.xlsx')

In [776]:
join = pd.merge(df,df_sum,how='left',on='CheckName')
join

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt_x,FormRpt_y
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...
13364,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,0,1


In [777]:
join = join.drop(columns=['FormRpt_x'])
join

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt_y
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
13364,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,NaN,1


In [778]:
join = join[join['FormRpt_y'] > 0]
join

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt_y
65,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
13364,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,NaN,1


In [779]:
combine_FormOID = join[~join['FormOID'].isna()]

combine_FormOID = combine_FormOID.drop_duplicates(subset=['CheckName','FormOID'])

combine_FormOID['FormOID'] = combine_FormOID['FormOID'].astype(str)

combine_FormOID = combine_FormOID.groupby('CheckName')['FormOID'].apply('|'.join).reset_index()

combine_FormOID

,CheckName,FormOID
0,Chk_AE_MIX_AEACNOTH_CM1_01_a_CF,AE_MIX
...,...,...
102,Derive_MinDate_Z_CMEDAT4_b_CF,CM_POST


In [780]:
combine_FormOID.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\combine_FormOID.xlsx',index=False)

In [781]:
join_1 = pd.merge(join,combine_FormOID,how='left',on='CheckName')
join_1

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt_y,FormOID_y
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,2,CM_PST|MH_DHH
...,...,...,...,...,...,...,...,...,...,...,...
534,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,...,NaN,NaN,NaN,1,CM_POST


In [782]:
join_1.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\join_1.xlsx',index=False)

In [783]:
join_1_x = join_1[join_1['FormOID_y'].str.contains(r'[@#&$%+-/|*]')]
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRpt_y,FormOID_y
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,2,CM_PST|MH_DHH
...,...,...,...,...,...,...,...,...,...,...,...
460,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,...,NaN,NaN,NaN,1,CM_POST|DS_EOS


In [784]:
#join_1_x['NoFormRptNumber'] = join_1_x[(~join_1_x['FormOID_x'].isna()) & (join_1_x['FormOID_x'] != 'CM_PST') & (join_1_x['FormOID_x'] != 'AE_MIX') & (join_1_x['FormOID_x'] != 'CM_POST') &
                              #(join_1_x['FormRepeatNumber'].isna())]
# -1 means FormOID is blank
# 99 menas non-relevant
# 1 means find FormRptNum not filled
# 0 means not found
# 99 menas non-relevant
def FormRptNumIsEmpty(x,y):
    if pd.isna(x):
        result = 0
    elif ~pd.isna(x) and x != 'CM_PST' and x != 'CM_PST' and x != 'AE_MIX' and x != 'CM_POST' and pd.isna(y):
        result = 1
    elif ~pd.isna(x) and x != 'CM_PST' and x != 'CM_PST' and x != 'AE_MIX' and x != 'CM_POST'and ~pd.isna(y):
        result = 0
    else:
        result = 0
    return result

join_1_x['FormRptNumIsEmpty'] = join_1_x.apply(lambda x: FormRptNumIsEmpty(x['FormOID_x'], x['FormRepeatNumber']), axis=1)
join_1_x

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,FormRepeatNumber,FolderRepeatNumber,FormRpt_y,FormOID_y,FormRptNumIsEmpty
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,2,CM_PST|MH_DHH,0
...,...,...,...,...,...,...,...,...,...,...,...
460,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,...,NaN,NaN,1,CM_POST|DS_EOS,0


In [785]:
join_1_x.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\FormRptNumIsEmpty.xlsx',index=False)

In [786]:
FormRptNumIsEmpty_Sum = join_1_x.groupby('CheckName').sum()
FormRptNumIsEmpty_Sum

,StepOrdinal,RecordPosition,FormRepeatNumber,FolderRepeatNumber,FormRpt_y,FormRptNumIsEmpty
CheckName,,,,,,
Chk_AE_MIX_AEACN_DW1_04,171,0.0,0.0,0.0,54,0
...,...,...,...,...,...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,91,0.0,0.0,0.0,26,2


In [787]:
FormRptNumIsEmpty_Sum = pd.DataFrame(FormRptNumIsEmpty_Sum,columns=['FormRptNumIsEmpty'])
FormRptNumIsEmpty_Sum

,FormRptNumIsEmpty
CheckName,
Chk_AE_MIX_AEACN_DW1_04,0
...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2


In [788]:
FormRptNumIsEmpty_Sum = FormRptNumIsEmpty_Sum.rename(columns={'FormRptNumIsEmpty': 'FormRptNumIsEmpty_Sum'})
FormRptNumIsEmpty_Sum

,FormRptNumIsEmpty_Sum
CheckName,
Chk_AE_MIX_AEACN_DW1_04,0
...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2


In [789]:
join_1_x = pd.merge(join_1_x,FormRptNumIsEmpty_Sum,how='left',on='CheckName')
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,FolderRepeatNumber,FormRpt_y,FormOID_y,FormRptNumIsEmpty,FormRptNumIsEmpty_Sum
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,2,CM_PST|MH_DHH,0,2
...,...,...,...,...,...,...,...,...,...,...,...
119,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,...,NaN,1,CM_POST|DS_EOS,0,0


In [790]:
join_1_x = join_1_x.drop(columns=['FormRpt_y','FormRptNumIsEmpty'])
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormOID_y,FormRptNumIsEmpty_Sum
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,2
...,...,...,...,...,...,...,...,...,...,...,...
119,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,...,NaN,NaN,NaN,CM_POST|DS_EOS,0


In [791]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 10)

def flag(x):
    if x > 0:
        result = 'Yes'
    elif x == 0:
        result = 'No'
    else:
        result = 'n/a'
    return result

join_1_x['Is_EmptyFormRptNum_Existed'] = join_1_x['FormRptNumIsEmpty_Sum'].apply(flag)
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,FormRepeatNumber,FolderRepeatNumber,FormOID_y,FormRptNumIsEmpty_Sum,Is_EmptyFormRptNum_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,CM_PST|MH_DHH,2,Yes
1,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2,NaN,Other,$30,...,NaN,NaN,CM_PST|MH_DHH,2,Yes
2,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,3,IsEqualTo,NaN,NaN,...,NaN,NaN,CM_PST|MH_DHH,2,Yes
3,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,4,NaN,NaN,StandardValue,...,NaN,NaN,CM_PST|MH_DHH,2,Yes
4,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,5,IsNotEmpty,NaN,NaN,...,NaN,NaN,CM_PST|MH_DHH,2,Yes
...,...,...,...,...,...,...,...,...,...,...,...
115,Chk_CM_POST_REASEND4_01,1,NaN,NaN,UserValue,...,NaN,NaN,CM_POST|DS_EOS,0,No
116,Chk_CM_POST_REASEND4_01,2,IsEmpty,NaN,NaN,...,NaN,NaN,CM_POST|DS_EOS,0,No
117,Chk_CM_POST_REASEND4_01,3,NaN,NaN,StandardValue,...,0.0,0.0,CM_POST|DS_EOS,0,No
118,Chk_CM_POST_REASEND4_01,4,IsNotEmpty,NaN,NaN,...,NaN,NaN,CM_POST|DS_EOS,0,No


In [792]:
join_1_x = join_1_x.drop(columns=['FormRptNumIsEmpty_Sum'])
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormOID_y,Is_EmptyFormRptNum_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
1,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2,NaN,Other,$30,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
2,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,3,IsEqualTo,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
3,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,4,NaN,NaN,StandardValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
4,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,5,IsNotEmpty,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
...,...,...,...,...,...,...,...,...,...,...,...
115,Chk_CM_POST_REASEND4_01,1,NaN,NaN,UserValue,...,NaN,NaN,NaN,CM_POST|DS_EOS,No
116,Chk_CM_POST_REASEND4_01,2,IsEmpty,NaN,NaN,...,NaN,NaN,NaN,CM_POST|DS_EOS,No
117,Chk_CM_POST_REASEND4_01,3,NaN,NaN,StandardValue,...,NaN,0.0,0.0,CM_POST|DS_EOS,No
118,Chk_CM_POST_REASEND4_01,4,IsNotEmpty,NaN,NaN,...,NaN,NaN,NaN,CM_POST|DS_EOS,No


In [793]:
join_1_x = join_1_x[join_1_x['Is_EmptyFormRptNum_Existed'] == 'Yes']
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,...,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormOID_y,Is_EmptyFormRptNum_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
1,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2,NaN,Other,$30,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
2,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,3,IsEqualTo,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
3,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,4,NaN,NaN,StandardValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
4,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,5,IsNotEmpty,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
5,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,6,NaN,NaN,CodedValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
6,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,7,NaN,NHL,$10,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
7,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,8,IsNotEqualTo,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
8,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,9,NaN,NaN,StandardValue,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes
9,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,10,IsNotEmpty,NaN,NaN,...,NaN,NaN,NaN,CM_PST|MH_DHH,Yes


In [794]:
join_1_x.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_CheckIfFormRptNumIsEntered.xlsx',index=False)